## Mounting google drive to colab

In [0]:
# Comment this section if using loacal runtime
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## Importing required libraries

In [0]:
import os
from tqdm import tqdm

import numpy as np
#import pyaudio

import wave
from scipy.io import wavfile
import librosa
import scipy.signal as ss



In [0]:
windowLength = 256
win          = np.hamming(windowLength)
overlap      = round(0.75 * windowLength)
ffTLength    = windowLength
inputFs      = 48e3
fs           = 16e3
numFeatures  = ffTLength//2 + 1
numSegments  = 8

### Function to convert sampling rate

In [0]:
def convert_sampling_rate(audio_file_dir, save_audio_dir, sampling_rate):
    audio , fs = librosa.load(audio_file_dir, sr = sampling_rate)
    if os.path.exists(save_audio_dir):
        librosa.output.write_wav(save_audio_dir+'/'+audio_file_dir.split('/')[1], audio, 16000)
    else:
        os.mkdir(save_audio_dir)
    
for file in tqdm(os.listdir("Nonspeech")):
    convert_sampling_rate("Nonspeech"+'/'+file, 'Nonspeech_16KHz', 16000)

### Function ave audio data to audio file

In [0]:
chunk = 1024  # Record in chunks of 1024 samples
#sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1
fs = 16000  # Record at 44100 samples per second
seconds = 1

#p = pyaudio.PyAudio()

def save_wav_file(filename, audio_data):
    wf = wave.open(filename, 'wb')
    wf.setnchannels(channels)
    wf.setsampwidth(2)
    wf.setframerate(fs)
    wf.writeframes(b''.join(audio_data))
    wf.close()

### Function to get audio data of specific chunk

In [0]:
def dissect_wav_file(read_dir='', save_dir=''):
    
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        print("Directory " , save_dir , " Created ")
    else:    
        print("Directory " , save_dir ,  "already exists")
        
    #list the folder to expect inside read_dir var
    #folders = ['clean', 'noisy']
    folders = ['8D1', '8D2', '8D3', '8D4', '8D5', '8D6', '8D7', '8D8', '8D9', '8D10']
    for folder in folders:
        
        if not os.path.exists(f'{save_dir}/{folder}'):
            os.mkdir(f'{save_dir}/{folder}')
            print("Directory " , folder ,  "Created ")
        else:    
            print("Directory " , folder ,  "already exists")
            
        path = f'{read_dir}/{folder}'
        files = os.listdir(path)
        print('#Files: ', len(files))
        
        i = 1
    
        for file in tqdm(files):
            try:
                if file.endswith('.wav'):
                    fs, audio_data = wavfile.read(read_dir+'/'+folder+'/'+file)
                    audio_data = audio_data[8000:40576]
                    if len(audio_data)==32576:
                        save_wav_file(f'{save_dir}/{folder}{i}.wav', audio_data)
                        i = i+1
                    
            except FileNotFoundError as fnf_error:
                print(fnf_error)


In [0]:
dissect_wav_file('drive/My Drive/Project/Silence', 'drive/My Drive/Project/Silence/')

 25%|██▌       | 2/8 [00:00<00:00, 17.37it/s]

Directory  drive/My Drive/Project/Silence/ already exists
Directory  8D1 already exists
#Files:  8


 25%|██▌       | 2/8 [00:00<00:00, 14.49it/s]

Directory  8D2 already exists
#Files:  8


 38%|███▊      | 3/8 [00:00<00:00, 18.28it/s]

...

 25%|██▌       | 2/8 [00:00<00:00, 19.85it/s]

Directory  8D9 already exists
#Files:  8


 25%|██▌       | 2/8 [00:00<00:00, 19.41it/s]

Directory  8D10 already exists
#Files:  8


100%|██████████| 8/8 [00:00<00:00, 21.03it/s]


### Function to get 8 segment of data

In [0]:
def get_8segments_xmag(stft_datastore, num_features=129, num_timestamps=510, num_segments=8):
    
    stft_datastore = stft_datastore.reshape(-1,num_features,num_timestamps)
        
    data_stft_segments = []
        
    for stft in tqdm(stft_datastore):
      
        data_stft = stft
        data_stft = np.hstack((data_stft[:,0:num_segments - 1], data_stft))
        
        stft_segments = np.zeros((num_segments, num_features, data_stft.shape[1] - num_segments + 1))
        
        for index in (range(0,data_stft.shape[1] - num_segments + 1)):
            stft_segments[:,:,index] = data_stft[:,index:index + num_segments].T
        
        for i in range(0,num_timestamps, 17):
          data_stft_segments.append(stft_segments[:,:,i:i+17])
        
    data_stft_segments = np.array(data_stft_segments)
    
 
            
    return data_stft_segments

### Function to get 1 segment of data

In [0]:
def get_1segment_ymag(stft_datastore,num_features=129, num_timestamps=510):
    stft_datastore = stft_datastore.reshape(-1,num_features,num_timestamps)
        
    data_stft_segments = []
    
    #data_stft_segments = [stft[:,index:index+17] for stft in tqdm(stft_datastore) for index in range(0,num_timestamps,17)]
        
    for stft in tqdm(stft_datastore):
        for index in range(0,num_timestamps,17):
            
            data_stft_segments.append(stft[:,index:index+17])

    data_stft_segments = np.array(data_stft_segments)
    data_stft_segments = data_stft_segments.reshape(data_stft_segments.shape[0],1,num_features, 17)
    
    return data_stft_segments

### Function to create magnitude and phase dataframe from  audio data

In [0]:
def create_audio_magnitude_phase_dataframe(dir_name='',batch_size=0, batch_pos=0, folders=['clean', 'noisy']):
    data_list = []
    if os.path.exists(dir_name):
        for folder in folders:
            for file in tqdm(os.listdir(dir_name+'/'+folder)[batch_pos:batch_pos+batch_size]):
                fs, data = wf.read(dir_name+'/'+folder+'/'+file)
                STFT = ss.stft(data,fs=fs,window='hamm',noverlap=overlap,nfft=ffTLength)
                time_vs_freq = STFT[2]
                data_list.append(time_vs_freq)    

    else:    
        print("Directory " , dir_name ,  "doesnot exist")
    
    
    x_mag = np.abs(data_list[0:batch_size])
    x_phase = np.angle(data_list[0:batch_size])
    y_mag = 0#np.abs(data_list[batch_size:(batch_size*2)])
    y_phase = 0#np.angle(data_list[batch_size:(batch_size*2)])
        
    return x_mag, x_phase, y_mag, y_phase

In [0]:
folders=['Silence']
x_mag, x_phase, y_mag, y_phase = create_audio_magnitude_phase_dataframe(dir_name='drive/My Drive/Project/Nonspeech',
                                     batch_size=80,
                                     batch_pos=0,
                                     folders=folders)


100%|██████████| 80/80 [00:00<00:00, 229.77it/s]


## Create Noisy Speech from Drone Noise

In [0]:
def create_noisy_speech_from_drone_noise(clean_data, noise_data):
    index = np.random.randint(0,len(drone_noise_data)-len(clean_data),1)
    index = np.asscalar(index)
    
    noise_segment = noise_data[index:index + len(clean_data)]
    
    clean_power = sum(clean_data.astype('int32')**2)
    noise_power = sum(drone_noise_data.astype('int32')**2)
    
    noisy_speech   = clean_data + np.sqrt(clean_power/noise_power) * noise_segment
    
    return noisy_speech
 

##  Store Data in Segments


In [0]:
x_mag = np.load("drive/My Drive/Project/Train/x_mag_2000batch1_drone_full_noise.npy")

In [0]:
def get_8segments_xmag(stft_datastore, num_features=129, num_timestamps=510, num_segments=8):
    
    stft_datastore = stft_datastore.reshape(-1,num_features,num_timestamps)
        
    data_stft_segments = []
        
    for stft in tqdm(stft_datastore):
      
        data_stft = stft
        data_stft = np.hstack((data_stft[:,0:num_segments - 1], data_stft))
        
        stft_segments = np.zeros((num_segments, num_features, data_stft.shape[1] - num_segments + 1))
        for index in (range(0,data_stft.shape[1] - num_segments + 1)):
            stft_segments[:,:,index] = data_stft[:,index:index + num_segments].T
        
        for i in range(0,num_timestamps, 17):
          data_stft_segments.append(stft_segments[:,:,i:i+17])
        
    data_stft_segments = np.array(data_stft_segments)
    
 
            
    return data_stft_segments

In [0]:
index = [1]
for ii in (range(1,2,1)):
    x_mag = np.load(f'drive/My Drive/Project/Train/x_mag_2000batch{ii}_drone_full_noise.npy')
    print(f'\nload drive/My Drive/Project/Train/x_mag_2000batch{ii}_drone_full_noise.npy')
  
    for jj in range(0,4,1):
      
      print(f'\n Loading data range : {500*jj}:{500*jj+500}')
      x_mag_temp = get_8segments_xmag(x_mag[(500*jj):(500*jj+500)])
      
      print(f'saving file to drive/My Drive/Project/Train/x_mag_500batch{index[ii-1]+jj}_drone_full_noise.npy')
      file_name = f'drive/My Drive/Project/Train/x_mag_500batch{index[ii-1]+jj}_drone_full_noise.npy'
      np.save(file_name, x_mag_temp)
      x_mag_temp = 0
      print(f'\nsaved file to drive/My Drive/Project/Train/x_mag_500batch{index[ii-1]+jj}_drone_full_noise.npy')
      
    x_mag = 0


  5%|▍         | 24/500 [00:00<00:02, 231.87it/s]


load drive/My Drive/Project/Train/x_mag_2000batch1_drone_full_noise.npy

 Loading data range : 0:500


100%|██████████| 500/500 [00:02<00:00, 206.53it/s]


saving file to drive/My Drive/Project/Train/x_mag_500batch1_drone_full_noise.npy


  4%|▍         | 19/500 [00:00<00:02, 187.15it/s]

...

saving file to drive/My Drive/Project/Train/x_mag_500batch3_drone_full_noise.npy


  3%|▎         | 14/500 [00:00<00:03, 139.57it/s]


saved file to drive/My Drive/Project/Train/x_mag_500batch3_drone_full_noise.npy

 Loading data range : 1500:2000


100%|██████████| 500/500 [00:02<00:00, 180.45it/s]


saving file to drive/My Drive/Project/Train/x_mag_500batch4_drone_full_noise.npy

saved file to drive/My Drive/Project/Train/x_mag_500batch4_drone_full_noise.npy


# Y data store

In [0]:
import numpy as np
from tqdm import tqdm
y_mag = np.load("drive/My Drive/Project/Train/y_mag_2000batch3.npy")

In [0]:
def get_1segment_ymag(stft_datastore,num_features=129, num_timestamps=510):
    stft_datastore = stft_datastore.reshape(-1,num_features,num_timestamps)
        
    data_stft_segments = []
    
    #data_stft_segments = [stft[:,index:index+17] for stft in tqdm(stft_datastore) for index in range(0,num_timestamps,17)]
        
    for stft in tqdm(stft_datastore):
        for index in range(0,num_timestamps,17):
            
            data_stft_segments.append(stft[:,index:index+17])

    data_stft_segments = np.array(data_stft_segments)
    data_stft_segments = data_stft_segments.reshape(data_stft_segments.shape[0],1,num_features, 17)
    
    return data_stft_segments

In [0]:
y_mag1 = get_1segment_ymag(y_mag[0:500])
y_mag2 = get_1segment_ymag(y_mag[500:1000])
y_mag3 = get_1segment_ymag(y_mag[1000:1500])
y_mag4 = get_1segment_ymag(y_mag[1500:2000])

100%|██████████| 500/500 [00:00<00:00, 44614.56it/s]


In [0]:
np.save("drive/My Drive/Project/Train/y_mag_500batch9.npy", y_mag1)
np.save("drive/My Drive/Project/Train/y_mag_500batch10.npy", y_mag2)
np.save("drive/My Drive/Project/Train/y_mag_500batch11.npy", y_mag3)
np.save("drive/My Drive/Project/Train/y_mag_500batch12.npy", y_mag4)

In [0]:
y_mag1=0
y_mag2=0
y_mag3=0
y_mag4=0